In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, make_scorer, r2_score
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict, GridSearchCV
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('datasets\\output\\train_clear.csv', index_col=0)

In [3]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Pave,Reg,Lvl,AllPub,...,0,Un,UnKn,Othr,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Pave,Reg,Lvl,AllPub,...,0,Un,UnKn,Othr,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,Pave,IR1,Lvl,AllPub,...,0,Un,UnKn,Othr,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,Pave,IR1,Lvl,AllPub,...,0,Un,UnKn,Othr,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,Pave,IR1,Lvl,AllPub,...,0,Un,UnKn,Othr,0,12,2008,WD,Normal,250000


In [4]:
test = pd.read_csv('datasets\\output\\test_clear.csv', index_col=0)

In [5]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,Pave,Reg,Lvl,AllPub,...,120,0,Un,MnPrv,Othr,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,Pave,IR1,Lvl,AllPub,...,0,0,Un,UnKn,Othr,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,Pave,IR1,Lvl,AllPub,...,0,0,Un,MnPrv,Othr,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,Pave,IR1,Lvl,AllPub,...,0,0,Un,UnKn,Othr,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,Pave,IR1,HLS,AllPub,...,144,0,Un,UnKn,Othr,0,1,2010,WD,Normal


In [6]:
result_id = test['Id']

In [7]:
for i in test, train:
    i.drop('Id', axis=1, inplace=True)

Отберем в отдельный список названия категориальных признаков

In [8]:
categorical = train.select_dtypes(['object']).columns

In [9]:
for i in categorical:
    train[i] = train[i].astype('category')
    test[i] = test[i].astype('category')

Отделим целевой признак от остальных:

In [10]:
features = train.drop('SalePrice', axis=1)

In [11]:
target = train['SalePrice']

In [12]:
X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.2, random_state=42)

In [13]:
X_val.shape, X_train.shape, y_train.shape, y_val.shape

((236, 79), (944, 79), (944,), (236,))

### Используем LGBMRegressor

In [14]:
model = LGBMRegressor(random_state=42)

Оценивать качество модели будем по метрике rmse, кроссвалидацию проведем встроеным кросвалидатором lgbm

In [15]:
def calc_rmse(target, predict):
    return (mean_squared_error(target, predict))**0.5

Подготовим метрику

In [16]:
rmse = make_scorer(calc_rmse, greater_is_better=False)

Подготовим функцию для проведения кроссвалидации:

In [17]:
def cross_val_mod(model, X, y):
    score = cross_val_score(model, X, y, cv=5, scoring=rmse)
    print(-score)
    print(f'RMSE модели на кроссвалидации равно - {(-score).mean()}')

In [18]:
cross_val_mod(model, X_train, y_train)

[21542.15298484 31495.49120232 22226.79599231 26285.20133845
 30218.02249748]
RMSE модели на кроссвалидации равно - 26353.532803079328


Проверим RMSE предсказаний на валидационной выборке

In [19]:
def val_score(model):
    model.fit(X_train, y_train)
    predict = model.predict(X_val)
    print(f'RMSE на валидационной выборке равно {calc_rmse(y_val, predict)}')
    print(f'R2 на валидационной выборке равно {r2_score(y_val, predict)}')

In [20]:
val_score(model)

RMSE на валидационной выборке равно 38234.91854471975
R2 на валидационной выборке равно 0.788887624048455


Подберем гиперпараметры:

In [21]:
params = {
    'boosting_type': ['gbdt'],
    'n_estimators': range(150, 181, 5),
    'max_depth': (3, 7),
    'reg_alpha': [0.1],
    'reg_lambda': [0.1]
}

In [22]:
grid = GridSearchCV(model, params, cv=5, scoring=rmse)

In [23]:
%%time
best_model = grid.fit(X_train, y_train)

Wall time: 17.1 s


In [24]:
best_model.best_params_

{'boosting_type': 'gbdt',
 'max_depth': 3,
 'n_estimators': 180,
 'reg_alpha': 0.1,
 'reg_lambda': 0.1}

In [25]:
%%time
cross_val_mod(best_model.best_estimator_, X_train, y_train)

[20795.86826731 30887.06409925 20772.53931611 26592.5896641
 28725.15569776]
RMSE модели на кроссвалидации равно - 25554.643408904736
Wall time: 1.15 s


In [26]:
%%time
val_score(best_model.best_estimator_)

RMSE на валидационной выборке равно 36244.733245512296
R2 на валидационной выборке равно 0.810293081149893
Wall time: 208 ms


Немного улучшилось, продолжим подбор гиперпараметров

In [86]:
params2 = {
    'n_estimators': range(190, 210, 2),
    'max_depth': range(3, 8),
    'reg_alpha': [0.1],
    'reg_lambda': [0.1],
    'num_leaves': range(10, 41, 5)
}

In [87]:
grid2 = GridSearchCV(model, params2, cv=5, scoring=rmse, verbose=3)

In [88]:
%%time
best_model_2 = grid2.fit(X_train, y_train)

Fitting 5 folds for each of 350 candidates, totalling 1750 fits
[CV] max_depth=3, n_estimators=190, num_leaves=10, reg_alpha=0.1, reg_lambda=0.1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=3, n_estimators=190, num_leaves=10, reg_alpha=0.1, reg_lambda=0.1, score=-20935.804, total=   0.3s
[CV] max_depth=3, n_estimators=190, num_leaves=10, reg_alpha=0.1, reg_lambda=0.1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  max_depth=3, n_estimators=190, num_leaves=10, reg_alpha=0.1, reg_lambda=0.1, score=-30888.648, total=   0.4s
[CV] max_depth=3, n_estimators=190, num_leaves=10, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth=3, n_estimators=190, num_leaves=10, reg_alpha=0.1, reg_lambda=0.1, score=-20626.230, total=   0.2s


...

[CV]  max_depth=6, n_estimators=190, num_leaves=10, reg_alpha=0.1, reg_lambda=0.1, score=-25940.542, total=   0.2s
[CV] max_depth=6, n_estimators=190, num_leaves=10, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth=6, n_estimators=190, num_leaves=10, reg_alpha=0.1, reg_lambda=0.1, score=-28907.383, total=   0.2s
[CV] max_depth=6, n_estimators=190, num_leaves=15, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth=6, n_estimators=190, num_leaves=15, reg_alpha=0.1, reg_lambda=0.1, score=-21664.422, total=   0.5s
[CV] max_depth=6, n_estimators=190, num_leaves=15, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth=6, n_estimators=190, num_leaves=15, reg_alpha=0.1, reg_lambda=0.1, score=-31871.411, total=   0.2s
[CV] max_depth=6, n_estimators=190, num_leaves=15, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth=6, n_estimators=190, num_leaves=15, reg_alpha=0.1, reg_lambda=0.1, score=-22296.441, total=   0.2s
[CV] max_depth=6, n_estimators=190, num_leaves=15, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth

[CV]  max_depth=6, n_estimators=192, num_leaves=20, reg_alpha=0.1, reg_lambda=0.1, score=-21638.815, total=   0.3s
[CV] max_depth=6, n_estimators=192, num_leaves=20, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth=6, n_estimators=192, num_leaves=20, reg_alpha=0.1, reg_lambda=0.1, score=-31543.529, total=   0.4s
[CV] max_depth=6, n_estimators=192, num_leaves=20, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth=6, n_estimators=192, num_leaves=20, reg_alpha=0.1, reg_lambda=0.1, score=-22586.290, total=   0.3s
[CV] max_depth=6, n_estimators=192, num_leaves=20, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth=6, n_estimators=192, num_leaves=20, reg_alpha=0.1, reg_lambda=0.1, score=-25799.419, total=   0.2s
[CV] max_depth=6, n_estimators=192, num_leaves=20, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth=6, n_estimators=192, num_leaves=20, reg_alpha=0.1, reg_lambda=0.1, score=-29710.706, total=   0.3s
[CV] max_depth=6, n_estimators=192, num_leaves=25, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth

[CV]  max_depth=6, n_estimators=194, num_leaves=25, reg_alpha=0.1, reg_lambda=0.1, score=-22193.185, total=   0.3s
[CV] max_depth=6, n_estimators=194, num_leaves=25, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth=6, n_estimators=194, num_leaves=25, reg_alpha=0.1, reg_lambda=0.1, score=-26725.186, total=   0.2s
[CV] max_depth=6, n_estimators=194, num_leaves=25, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth=6, n_estimators=194, num_leaves=25, reg_alpha=0.1, reg_lambda=0.1, score=-29829.429, total=   0.2s
[CV] max_depth=6, n_estimators=194, num_leaves=30, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth=6, n_estimators=194, num_leaves=30, reg_alpha=0.1, reg_lambda=0.1, score=-21799.384, total=   0.2s
[CV] max_depth=6, n_estimators=194, num_leaves=30, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth=6, n_estimators=194, num_leaves=30, reg_alpha=0.1, reg_lambda=0.1, score=-31335.908, total=   0.3s
[CV] max_depth=6, n_estimators=194, num_leaves=30, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth

[CV]  max_depth=6, n_estimators=196, num_leaves=30, reg_alpha=0.1, reg_lambda=0.1, score=-29813.593, total=   0.3s
[CV] max_depth=6, n_estimators=196, num_leaves=35, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth=6, n_estimators=196, num_leaves=35, reg_alpha=0.1, reg_lambda=0.1, score=-21731.447, total=   0.2s
[CV] max_depth=6, n_estimators=196, num_leaves=35, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth=6, n_estimators=196, num_leaves=35, reg_alpha=0.1, reg_lambda=0.1, score=-31381.057, total=   0.2s
[CV] max_depth=6, n_estimators=196, num_leaves=35, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth=6, n_estimators=196, num_leaves=35, reg_alpha=0.1, reg_lambda=0.1, score=-23197.326, total=   0.2s
[CV] max_depth=6, n_estimators=196, num_leaves=35, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth=6, n_estimators=196, num_leaves=35, reg_alpha=0.1, reg_lambda=0.1, score=-26527.436, total=   0.3s
[CV] max_depth=6, n_estimators=196, num_leaves=35, reg_alpha=0.1, reg_lambda=0.1 
[CV]  max_depth

KeyboardInterrupt: 

In [89]:
best_model_2.best_params_

{'max_depth': 3,
 'min_child_samples': 14,
 'n_estimators': 194,
 'reg_alpha': 0.1,
 'reg_lambda': 0.1}

In [90]:
%%time
cross_val_mod(best_model_2.best_estimator_, X_train, y_train)

[20507.75763199 30380.08179584 21001.86966348 26481.68510608
 26996.88171076]
RMSE модели на кроссвалидации равно - 25073.655181631508
Wall time: 1.24 s


In [91]:
%%time
val_score(best_model_2.best_estimator_)

RMSE на валидационной выборке равно 37402.46686227322
R2 на валидационной выборке равно 0.7979802410520125
Wall time: 209 ms
